# Linear Advection — Numerical Schemes (Periodic BC)
This notebook studies explicit finite‑difference schemes for the 1D linear advection equation:

$$u_t + a(x)\,u_x = 0,\qquad x\in[0,\pi],\ t\ge0,$$
with periodic boundary conditions. We compare **Upwind**, **Lax–Friedrichs**, and **Lax–Wendroff** (constant speed).

**Exact solutions** are provided for:

- $a(x)=1$ (translation)
- $a(x)=\sin x$ (via characteristics: $\tan(x/2)=\tan(x_0/2)\,e^{t}$).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from dataclasses import dataclass
from typing import Callable, Tuple, Dict, Optional
PI = np.pi

def periodic_wrap(x, L):
    return np.mod(x, L)

def roll_periodic(u, k):
    return np.roll(u, k)

def a_speed(x, ind):
    if ind == 0:
        return np.ones_like(x)
    elif ind == 1:
        return np.sin(x)
    else:
        raise ValueError("ind must be 0 or 1")

def u0(x):
    return np.sin(2.0 * x)

def exact_solution(x, t, ind):
    if ind == 0:
        x0 = periodic_wrap(x - t, PI)
        return u0(x0)
    elif ind == 1:
        x0 = 2.0 * np.arctan(np.tan(0.5*x) * np.exp(-t))
        x0 = np.clip(x0, 0.0, PI)
        return u0(x0)
    else:
        raise ValueError("ind must be 0 or 1")

@dataclass
class Grid:
    J: int
    L: float = PI
    def __post_init__(self):
        self.mesh = np.linspace(0.0, self.L, self.J, endpoint=False)
        self.dx = self.mesh[1] - self.mesh[0]

def step_upwind(u, a, dt, dx):
    a_plus  = np.maximum(a, 0.0)
    a_minus = np.minimum(a, 0.0)
    u_ip1 = roll_periodic(u, -1)
    u_im1 = roll_periodic(u,  1)
    return u - (dt/dx) * ( a_plus * (u - u_im1) + a_minus * (u_ip1 - u) )

def step_lax_friedrichs(u, a, dt, dx):
    u_ip1 = roll_periodic(u, -1)
    u_im1 = roll_periodic(u,  1)
    return 0.5 * (u_ip1 + u_im1) - 0.5 * (dt/dx) * a * (u_ip1 - u_im1)

def step_lax_wendroff_const(u, a_const, dt, dx):
    c = a_const * dt / dx
    u_ip1 = roll_periodic(u, -1)
    u_im1 = roll_periodic(u,  1)
    return u - 0.5*c*(u_ip1-u_im1) + 0.5*(c**2)*(u_ip1 - 2*u + u_im1)

def run_scheme(scheme, J, dt, T, ind):
    grid = Grid(J=J)
    x = grid.mesh
    dx = grid.dx
    a = a_speed(x, ind)
    amax = np.max(np.abs(a))
    u = u0(x).copy()
    t = 0.0
    if scheme == "upwind":
        stepper = lambda u: step_upwind(u, a_speed(x, ind), dt, dx)
    elif scheme == "lax_friedrichs":
        stepper = lambda u: step_lax_friedrichs(u, a_speed(x, ind), dt, dx)
    elif scheme == "lax_wendroff":
        if ind != 0:
            raise ValueError("Lax–Wendroff here only for constant a (ind=0).")
        stepper = lambda u: step_lax_wendroff_const(u, 1.0, dt, dx)
    else:
        raise ValueError("Unknown scheme.")
    N = int(np.ceil(T/dt))
    for _ in range(N):
        u = stepper(u)
        t += dt
    u_ex = exact_solution(x, t, ind)
    err_inf = np.max(np.abs(u - u_ex))
    err_l2  = np.sqrt(np.mean((u - u_ex)**2))
    return {"x": x, "u": u, "u_exact": u_ex, "dx": dx, "t": t, "err_inf": err_inf, "err_l2": err_l2}

## Demo — Constant speed $a=1$
Run Upwind and Lax–Wendroff at $T=1$. Choose $\Delta t$ from a target CFL.

In [ ]:
T = 1.0
J = 100
grid = Grid(J)
dt = 0.9 * grid.dx  # CFL ~ 0.9 since a=1

out_up = run_scheme("upwind", J, dt, T, ind=0)
out_lw = run_scheme("lax_wendroff", J, dt, T, ind=0)

plt.figure(); plt.plot(out_up["x"], out_up["u"], 'o-', label='Upwind'); 
plt.plot(out_up["x"], out_up["u_exact"], '--', label='Exact'); plt.legend(); plt.xlabel("x"); plt.title("Upwind, a=1");
plt.show()

plt.figure(); plt.plot(out_lw["x"], out_lw["u"], 'o-', label='Lax–Wendroff'); 
plt.plot(out_lw["x"], out_lw["u_exact"], '--', label='Exact'); plt.legend(); plt.xlabel("x"); plt.title("Lax–Wendroff, a=1");
plt.show()

print("Errors (L2): Upwind =", out_up["err_l2"], " | Lax–Wendroff =", out_lw["err_l2"])

## Demo — Variable speed $a(x)=\sin x$
Compare Upwind vs. Lax–Friedrichs at $T=1$.

In [ ]:
J = 200
grid = Grid(J)
a = np.sin(grid.mesh)
dt = 0.9 * grid.dx / np.max(np.abs(a))

out_up = run_scheme("upwind", J, dt, 1.0, ind=1)
out_lf = run_scheme("lax_friedrichs", J, dt, 1.0, ind=1)

plt.figure(); plt.plot(out_up["x"], out_up["u"], 'o-', label='Upwind'); 
plt.plot(out_up["x"], out_up["u_exact"], '--', label='Exact'); plt.legend(); plt.xlabel("x"); plt.title("Upwind, a=sin x");
plt.show()

plt.figure(); plt.plot(out_lf["x"], out_lf["u"], 'o-', label='Lax–Friedrichs'); 
plt.plot(out_lf["x"], out_lf["u_exact"], '--', label='Exact'); plt.legend(); plt.xlabel("x"); plt.title("Lax–Friedrichs, a=sin x");
plt.show()

print("Errors (L2): Upwind =", out_up["err_l2"], " | Lax–Friedrichs =", out_lf["err_l2"])

## Convergence Study
We refine $J$ and pick $\Delta t$ from a target CFL.

In [ ]:
def convergence_study(scheme, J_list, ind, T, cfl_target=0.9):
    rows = []
    for J in J_list:
        grid = Grid(J)
        x = grid.mesh
        a = a_speed(x, ind)
        amax = np.max(np.abs(a))
        dt = cfl_target * grid.dx / amax if amax>0 else 1e-6
        out = run_scheme(scheme, J, dt, T, ind)
        rows.append({"J": J, "dx": out["dx"], "dt": dt, "err_L2": out["err_l2"]})
    return pd.DataFrame(rows).sort_values("J")

J_list = [25, 50, 100, 200]
df_up_const = convergence_study("upwind", J_list, ind=0, T=1.0)
df_lw_const = convergence_study("lax_wendroff", J_list, ind=0, T=1.0)
df_up_var   = convergence_study("upwind", J_list, ind=1, T=1.0)

print("Upwind, a=1:\n", df_up_const, "\n")
print("Lax–Wendroff, a=1:\n", df_lw_const, "\n")
print("Upwind, a=sin x:\n", df_up_var)